# RAG
* Container: `Data Science 3.0` (studio, python 3.10), `conda_python3` (notebook)

## 0. Install packages and Setup env

In [1]:
import sys

In [2]:
%load_ext autoreload
%autoreload 2
sys.path.append('../utils') # src 폴더 경로 설정

In [12]:
install_needed = True  # should only be True once

In [13]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U faiss-cpu
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.8/852.8 kB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.172.0-py2.py3-none-any.whl size=1160751 sha256=de87f1d007b09548bfc4474913d158c2ce68be4a17fec375884a8b652e1434d5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ba/42/85/8e88201cc1749af14596c0edeb7fa8b1562fb94e72f919a960
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.171.0
    Uninstalling sagemaker-2.171.0:
      Successfully uninstalled sagemaker-2.171.0
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━

### 1. SageMaker Endpoint Wrapper

### 1.1 SageMaker LLM_TEXT Wrapper

In [3]:
import json
import boto3
import numpy as np
from inference_utils import Prompter
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

In [63]:
prompter = Prompter("kullm")
params = {
      'do_sample': False,
      'max_new_tokens': 128,
      'temperature': 1.0,
      'top_k': 0,
      'top_p': 0.9,
      'return_full_text': False,
      'repetition_penalty': 1.1,
      'presence_penalty': None,
      'eos_token_id': 2
}

class KullmContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        '''
        입력 데이터 전처리 후에 리턴
        '''
        context, question = prompt.split("||SPEPERATOR||") 
        prompt = prompter.generate_prompt(question, context)

        print ("prompt", prompt)
        print ("model_kwargs", model_kwargs)
        
        payload = {
            'prompt': [prompt],
            'params': model_kwargs
        }
                           
        input_str = json.dumps(payload)
        
        return input_str.encode('utf-8')
    

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))              
        generated_text = response_json[0][0]["generated_text"]
        
        return generated_text    

In [64]:
aws_region = boto3.Session().region_name
LLMTextContentHandler = KullmContentHandler()
endpoint_name_text = "Kullm-polyglot-12-8b-v2-2023-07-10-05-48-59"
seperator = "||SPEPERATOR||"

In [65]:
llm_text = SagemakerEndpoint(
    endpoint_name=endpoint_name_text,
    region_name=aws_region,
    model_kwargs=params,    
    content_handler=LLMTextContentHandler,
)

### 1.2 SageMaker LLM_EMB Wrapper

In [66]:
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(self, texts: List[str], chunk_size: int=1) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        
        print("text size: ", len(texts))
        print("_chunk_size: ", _chunk_size)

        for i in range(0, len(texts), _chunk_size):
            
            #print (i, texts[i : i + _chunk_size])
            response = self._embedding_func(texts[i : i + _chunk_size])
            #print (i, response, len(response[0].shape))
            
            results.extend(response)
        return results

In [67]:
class KoSimCSERobertaContentHandler(EmbeddingsContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))
        ndim = np.array(response_json).ndim    
        
        if ndim == 4:
            # Original shape (1, 1, n, 768)
            emb = response_json[0][0][0]
            emb = np.expand_dims(emb, axis=0).tolist()
        elif ndim == 2:
            # Original shape (n, 1)
            emb = []
            for ele in response_json:
                e = ele[0][0]
                emb.append(e)
        else:
            print(f"Other # of dimension: {ndim}")
            emb = None
        return emb

In [68]:
LLMEmbHandler = KoSimCSERobertaContentHandler()
endpoint_name_emb = "KoSimCSE-roberta-2023-07-10-05-26-01"

In [69]:
llm_emb = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=endpoint_name_emb,
    region_name=aws_region,
    content_handler=LLMEmbHandler,
)

**Now, we can build an QA application. <span style="color:red">LangChain makes it extremly simple with following few lines of code</span>.**

## 4. Vector Store
FAISS Vector Store
- https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/faiss <BR>

OpenSearch
- https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/opensearch

In [14]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

### 4.1 load context files and build indexer

#### Increase csv field size

In [15]:
import csv
csv.field_size_limit(100000000)
csv.field_size_limit()

100000000

In [16]:
loader = CSVLoader(
    file_path="../dataset/quenstion_answer_ko.csv",
    source_column="Source",
    encoding="utf-8"
)
context_documents = loader.load()

In [17]:
len(context_documents), context_documents[5]

(8482,
 Document(page_content='Information: 이마트 창동점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. \n※ 임대매장(스타벅스, 식당 등)이용 무료주차    해당매장에서 무료주차 2시간 적용※ 계산대 출력영수증, 모바일영수증에     하단 바코드를 사용하여    신용카드 전용 사전무인정산기 정산후 출차    -케이엠파크:080-330-3600※ 전기차 충전소 위치: 옥외 주차장 8F     차지비,완속 3대,24:00~24:00, 고객센터 1600-4047 매장 전체 주차가능대수: 159대<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 5}))

#### Create Indexer
TextSplitter: https://js.langchain.com/docs/modules/indexes/text_splitters/examples/recursive_character <BR>
**It takes about 10 mins**

In [18]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=llm_emb,
    #text_splitter=CharacterTextSplitter(chunk_size=700, chunk_overlap=0),
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
)

## show documents
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#docs = text_splitter.split_documents(context_documents)

In [17]:
%%time
index = index_creator.from_loaders([loader])

text size:  8524
_chunk_size:  1
CPU times: user 3min 3s, sys: 5.28 s, total: 3min 8s
Wall time: 11min 31s


#### Save and Read index from local

In [18]:
# Save
index.vectorstore.save_local("../indexer/indexer")

In [19]:
# Read
index_ = FAISS.load_local("../indexer/indexer", llm_emb)

#### Check documents

In [20]:
#index.vectorstore.index_to_docstore_id
#index.vectorstore.docstore._dict
#context_documents[4].page_content
index_.index_to_docstore_id
index_.docstore._dict

{'7fb45520-e329-4ea8-a0b6-ac2fba34b16c': Document(page_content='Information: 이마트 창동점의 우편번호는 다음과 같습니다. \n132045<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 0}),
 'bd0ae9d7-bb52-40c4-bc2a-d98471cbb441': Document(page_content='Information: 이마트 창동점의 주소는 다음과 같습니다. \n서울 도봉구 노해로 65길 4<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 1}),
 '32b5d158-a374-4f51-bb56-89e259777055': Document(page_content='Information: 이마트 창동점의 전화번호는 다음과 같습니다. \n02-901-1234<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 2}),
 '943076e6-7f7b-4a25-b314-38cc4026f6d7': Document(page_content='Information: 이마트 창동점의 휴일은 다음과 같습니다/. \n2023-07-09, 2023-07-23, 2023-06-11, 2023-06-25, 2023-01-22<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 3}),
 'da60b507-d553-4d7b-8abb-5df27deb93b9': Document(page_content='Information: 이마트 창동점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. &lt;유료주차 운영 안내&gt;- 24시간운영- 무료회차 시간 30분이내- 기본요금: 10분당 1,000원- 30분 초과시 회차시간 포함 요

## 5. QnA

In [70]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

### 5.1 Query and Response

In [71]:
prompt_template = ''.join(["{context}", seperator, "{question}"])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(llm=llm_text, chain_type="stuff", prompt=PROMPT, verbose=False)

def get_similiar_docs(query, k=10, fetch_k=50, score=False, store=""):
    
    if score: 
        #similar_docs = index.vectorstore.similarity_search_with_score(query, k=k, fetch_k=fetch_k, filter=dict(source=store))
        similar_docs = index_.similarity_search_with_score(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=store)
        )
    else:
        #similar_docs = index.vectorstore.similarity_search(query, k=k, fetch_k=fetch_k, filter=dict(source=store))
        similar_docs = index_.similarity_search(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=store)
        )
        
    return similar_docs

def get_answer(query, store=""):
    
    similar_docs = get_similiar_docs(query, store=store)
    answer = chain.run(input_documents=similar_docs, question=query)
    
    return answer


In [72]:
%%time
question = "이마트 창동점 버스타고 가는 방법"
response = get_answer(question, store="이마트 창동점")

print (f'question: {question}')
print (f'response: {response}')

prompt 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
이마트 창동점 버스타고 가는 방법

### 입력:
Information: 이마트 창동점의 오시는 길은 다음과 같습니다. 
[지하철 이용시] 
창동역 2번출구 나오셔서 왼쪽방향

[버스 이용시]
마을버스 (도봉) 01번
초록(지선버스) 1120, 1133, 1157번

[승용차 이용시]
노원/태릉이용시 ▷노원역에서 승요차 이용시 창동교를 지나 창동지하차도지나 도봉등기소 앞 50m에서 U턴 실시
쌍문/수유미아에서 승용차 이용시▷ 쌍문역 앞 정의여중입구 사거리에서 노원방향으로 직진후 이마트 앞 창동지하차도 상단으로 U턴 실시<sep>이마트 창동점
Source: 이마트 창동점

Information: 네, 이마트 창동점에는 엄마밥상가 있고, 전화번호는 02-954-2349, 운영시간은 11:00~21:00 입니다.<sep>이마트 창동점
Source: 이마트 창동점

Information: 이마트 창동점의 전화번호는 다음과 같습니다. 
02-901-1234<sep>이마트 창동점
Source: 이마트 창동점

Information: 네, 이마트 창동점에는 비너스 슈(언더웨어)가 있고, 전화번호는 --, 운영시간은 10:00~22:00 입니다.<sep>이마트 창동점
Source: 이마트 창동점

### 응답:

model_kwargs {'do_sample': False, 'max_new_tokens': 128, 'temperature': 1.0, 'top_k': 0, 'top_p': 0.9, 'return_full_text': False, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}
question: 이마트 창동점 버스타고 가는 방법
response: 이마트 창동점에 가는 방법은 지하철, 버스, 승용차 등 다양한 교통수단을 이용할